# Create a Simple Reflex-Based Lunar Lander Agent

In this example, we will use Gymnasium, an environment to train agents via reinforcement learning (RL). We will not use RL here but just use the environment with a custom simple reflex-based agent.

## Install Gymnasium

The documentation for Gymnasium is available at https://gymnasium.farama.org/

Steps:
1. Create a new folder and open it with VS Code and install all needed Python Extensions in VS Code.
2. Create a new virtual environment (CTRL-Shift P Python Create Environment...)
3. I needed to install swig and the Python C++ headers on WSL2 via the terminal
    * `sudo apt install swig`
    * `sudo apt-get install python3-dev`
4. Install gymnasium with the needed extras

In [ ]:
%pip install -q swig
%pip install -q gymnasium[box2d,classic_control]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## The Lunar Lander Environment

The documentation of the environment is available at: https://gymnasium.farama.org/environments/box2d/lunar_lander/

* Performance Measure: A reward of -100 or +100 points for crashing or landing safely respectively. We do not use
  intermediate rewards here.

* Environment: This environment is a classic rocket trajectory optimization problem. A ship needs to land safely. The space is **continuous** with
  x and y coordinates in the range [-2.5, 2.5]. The landing pad is at coordinate (0,0).

* Actuators:  According to Pontryagin’s
  maximum principle, it is optimal to fire the engine at full throttle or turn it off. This is the reason why this environment has discrete actions: engine on or off. There are four discrete actions available:

    - 0: do nothing
    - 1: fire left orientation engine
    - 2: fire main engine
    - 3: fire right orientation engine

* Sensors: Each observation is an 8-dimensional vector: the coordinates of the lander in x & y, its linear velocities in x & y, its angle, its angular velocity, and two booleans that represent whether each leg is in contact with the ground or not.

Gymnasim environments are implemented as classes with a `make` method to create the environment, a `reset` method, and a `step` method to execute an action.
To use it with an agent function that expects percetps and returns an action, we need write glue code that connects the environment with the agent function.

In [ ]:
import gymnasium as gym

def run_episode(agent_function, max_steps=1000):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialize the environment
    env = gym.make("LunarLander-v3", render_mode="human")

    # Reset the environment to generate the first observation (use seed=42 in reset to get reproducible results)
    observation, info = env.reset()

    # run one episode
    for _ in range(max_steps):
        # call the agent function to select an action
        action = agent_function(observation)

        print (f"Obs: {observation} -> Action: {action}")

        # step: execute an action in the environment
        observation, reward, terminated, truncated, info = env.step(action)

        env.render()

        if terminated:
            print(f"Final Reward: {reward}")
            break

    env.close()
    return reward

Note: `env.render()` shows the environment when the notebook is locally run (e.g., in VScode). On Colab, you cannot see the environment because the code is run on a headless server (i.e., a server without a display). There are some workarounds you can google.

## Example: A Random Agent

We ranomly return one of the actions. The environment accepts the integers 0-3.


In [ ]:
import numpy as np

def random_agent_function(observation):
    """A random agent that selects actions uniformly at random. It ignores the observation."""
    return np.random.choice([0, 1, 2, 3], p=[0.25, 0.25, 0.25, 0.25])

run_episode(random_agent_function)

Obs: [ 0.00261602  1.4158009   0.2649613   0.21692756 -0.00302454 -0.0600177
  0.          0.        ] -> Action: 3
Obs: [ 0.00532541  1.4201152   0.27631906  0.19172978 -0.00833928 -0.1063038
  0.          0.        ] -> Action: 1
Obs: [ 0.00794077  1.4238322   0.26449743  0.16517884 -0.01127664 -0.05875236
  0.          0.        ] -> Action: 2
Obs: [ 0.01051807  1.427808    0.26091594  0.1766763  -0.01442842 -0.06304148
  0.          0.        ] -> Action: 2
Obs: [ 0.01310062  1.4318135   0.26143977  0.17798528 -0.01758347 -0.06310657
  0.          0.        ] -> Action: 0
Obs: [ 0.01568327  1.4352192   0.26144892  0.15131468 -0.0207381  -0.06309805
  0.          0.        ] -> Action: 1
Obs: [ 0.01819382  1.438039    0.25239557  0.1253063  -0.02207032 -0.0266465
  0.          0.        ] -> Action: 3
Obs: [ 0.02079897  1.4402632   0.26426166  0.09879182 -0.02577954 -0.07419139
  0.          0.        ] -> Action: 3
Obs: [ 0.02347727  1.4418842   0.2734331   0.07193802 -0.03132423 -

-100

## A Simple Reflex-Based Agent

To make the code easier to read, we use enumerations for actions (integers) and observations (index in the observation vector).

In [ ]:
from enum import Enum

class Act(Enum):
    LEFT = 1
    RIGHT = 3
    MAIN = 2
    NO_OP = 0

class Obs(Enum):
    X = 0
    Y = 1
    VX = 2
    VY = 3
    ANGLE = 4
    ANGULAR_VELOCITY = 5
    LEFT_LEG_CONTACT = 6
    RIGHT_LEG_CONTACT = 7


In [ ]:
def rocket_agent_function(observation):
    """A simple agent function."""

    # run the main thruster, if the lander is falling too fast
    if observation[Obs.VY.value] < -.3:
        return Act.MAIN.value

    return Act.NO_OP.value

run_episode(rocket_agent_function)

Obs: [-0.00437393  1.4156827  -0.44303828  0.21166696  0.00507498  0.10035469
  0.          0.        ] -> Action: 0
Obs: [-0.00874815  1.4198676  -0.44243908  0.18597212  0.01003398  0.09918963
  0.          0.        ] -> Action: 0
Obs: [-0.01312265  1.4234536  -0.44245487  0.15933126  0.01499002  0.09912983
  0.          0.        ] -> Action: 0
Obs: [-0.01749716  1.4264396  -0.44246945  0.13265845  0.01994542  0.09911741
  0.          0.        ] -> Action: 0
Obs: [-0.02187195  1.4288262  -0.442484    0.10598822  0.02490005  0.09910182
  0.          0.        ] -> Action: 0
Obs: [-0.02624683  1.4306129  -0.44249862  0.07931815  0.0298539   0.09908605
  0.          0.        ] -> Action: 0
Obs: [-0.03062191  1.4317999  -0.44251317  0.05264802  0.03480698  0.09907056
  0.          0.        ] -> Action: 0
Obs: [-0.03499718  1.4323874  -0.4425277   0.02597776  0.03975927  0.09905498
  0.          0.        ] -> Action: 0
Obs: [-3.9372541e-02  1.4323748e+00 -4.4254217e-01 -6.9261505e-0

-100

## Evaluating the Agent

Run the agent on 100 problems and report the average reward.

In [ ]:
import numpy as np

def run_episode_test(agent_function):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialise the environment
    env = gym.make("LunarLander-v3", render_mode=None)

    # Reset the environment to generate the first observation
    observation, info = env.reset()

    # run one episode (max. 1000 steps)
    for _ in range(1000):
        # call the agent to select an action
        action = agent_function(observation)

        # step (transition) through the environment with the action
        observation, reward, terminated, truncated, info = env.step(action)

        if terminated:
            break

    env.close()
    return reward

def run_episodes(agent_function, n=1000):
    """Run multiple episodes with the given agent and return the rewards for each episode."""
    return [run_episode_test(agent_function) for _ in range(n)]

rewards = run_episodes(rocket_agent_function)
print(rewards)

print(f"Average reward: {np.average(rewards)}")
print(f"Success rate: {np.sum(np.array(rewards) == 100)}/{len(rewards)}")

[-100, -100, -100, -100, -100, -100, -100, 100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,

This is not great!


## Implement A Better Reflex-Based Agent

Build a better that uses its right and left thrusters to land the craft (more) safely. Test your agent function using 100 problems.

In [ ]:
# Code goes here
def test_agent_function(observation):
    """A simple agent function."""

    # bật động cơ chính nếu tàu đang rơi quá nhanh
    if observation[Obs.VY.value] < -.3:
        return Act.MAIN.value

    if observation[Obs.ANGLE.value] <-.1: # bật động cơ trái, nếu tàu có góc nghiêng về bên trái quá -0.1 radian
        return Act.LEFT.value
    elif observation[Obs.ANGLE.value] > .1: # bật động cơ phải, nếu tàu có góc nghiêng về bên phải quá 0.1 radian
        return Act.RIGHT.value

    if observation[Obs.X.value] < -.5: # bật động cơ trái, nếu tàu bị lệch trái so với vị trí đáp
        return Act.LEFT.value
    elif observation[Obs.X.value] > .5: # bật động cơ phải, nếu tàu bị lệch phải so với vị trí đáp
        return Act.RIGHT.value

    return Act.NO_OP.value # Không hành động gì


In [ ]:
def run_episodes(agent_function, n=1000):
    """Run multiple episodes with the given agent and return the rewards for each episode."""
    return [run_episode_test(agent_function) for _ in range(n)]

rewards = run_episodes(test_agent_function, n = 1000)
print(f"Success Rate: {np.sum(np.array(rewards) == 100)}/{len(rewards)}")

Success Rate: 52/1000
